# Select & Remap Channels — Voila (MODULAR)

In [1]:

#%% Imports
import os, json, traceback
from pathlib import Path
import ipywidgets as W
from IPython.display import display as _ip_display, HTML, Markdown

try:
    import mne
    _reader = "mne"
except Exception:
    try:
        import pyedflib
        _reader = "pyedflib"
    except Exception:
        _reader = None

try:
    from ipyfilechooser import FileChooser
    _has_fc = True
except Exception:
    _has_fc = False

STATE = {}
def sdisplay(*objs):
    lst = STATE.setdefault("_refs", [])
    for obj in objs:
        lst.append(obj)
        _ip_display(obj)

def list_edf_files(folder: Path):
    if not folder.exists():
        return []
    return sorted([str(p) for p in folder.rglob("*.edf") if not p.name.startswith("._")])

def read_channels(path: Path):
    if _reader == "mne":
        try:
            raw = mne.io.read_raw_edf(str(path), preload=False, verbose="ERROR")
            return list(raw.ch_names)
        except Exception:
            return []
    elif _reader == "pyedflib":
        try:
            f = pyedflib.EdfReader(str(path))
            n = f.signals_in_file
            names = [f.getLabel(i) for i in range(n)]
            f.close()
            return names
        except Exception:
            return []
    return []

def pretty_json_head(d, n=3):
    from collections import OrderedDict
    keys = list(d.keys())[:n]
    return json.dumps(OrderedDict((k, d[k]) for k in keys), indent=2, ensure_ascii=False)

title = W.HTML("<h2>Select & Remap Channels (EDF)</h2>")
out = W.Output()

if _has_fc:
    chooser = FileChooser(os.getcwd())
    chooser.title = "<b>Choose your study folder</b>"
    chooser.show_only_dirs = True
else:
    chooser = W.Text(value=str(Path.home()), description="Folder:", layout=W.Layout(width="100%"))

files_sel = W.SelectMultiple(options=[], rows=8, description="EDF files:")
btn_use_all = W.Button(description="Use all", icon="check")
channels_sel = W.SelectMultiple(options=[], rows=12, description="Channels:")
btn_all = W.Button(description="Select all", icon="check")
btn_none = W.Button(description="Clear", icon="close")
rename_area = W.Textarea(value="{}", layout=W.Layout(width="100%", height="140px"))
reref_mode = W.Dropdown(options=[("Keep", "keep"), ("Average", "avg"), ("Custom", "custom")], value="keep", description="Re-ref:")
reref_custom = W.Text(value="", description="Custom refs:")
save_name = W.Text(value="channel_config.json", description="Filename:")
btn_save = W.Button(description="Save", icon="save", button_style="success")
preview = W.HTML("<pre>{}</pre>")

app = W.VBox([
    title,
    W.HBox([chooser]),
    W.HTML("<hr>"),
    W.HBox([files_sel, W.VBox([btn_use_all])]),
    W.HTML("<hr>"),
    W.HBox([channels_sel, W.VBox([btn_all, btn_none])]),
    W.HTML("<hr>"),
    W.HTML("<b>Rename mapping (JSON)</b>"),
    rename_area,
    W.HTML("<hr>"),
    W.HBox([reref_mode, reref_custom]),
    W.HTML("<hr>"),
    W.HTML("<b>Preview (first 3)</b>"),
    preview,
    W.HTML("<hr>"),
    W.HBox([save_name, btn_save]),
    out
])

def scan_folder(folder: Path):
    with out:
        print("📁 Selected Path:", folder)
    edfs = list_edf_files(folder)
    files_sel.options = edfs
    if edfs:
        files_sel.value = tuple(edfs)
        chs = read_channels(Path(edfs[0]))
        channels_sel.options = chs
        channels_sel.value = tuple(chs)
        rename_area.value = json.dumps({c: c for c in chs}, indent=2, ensure_ascii=False)
    else:
        channels_sel.options = []
        rename_area.value = "{}"

def on_choose(_=None):
    out.clear_output()
    folder = Path(chooser.selected_path) if _has_fc else Path(chooser.value).expanduser()
    scan_folder(folder)
    STATE["folder"] = str(folder)

def on_use_all(_): files_sel.value = tuple(files_sel.options)
def on_sel_all(_): channels_sel.value = tuple(channels_sel.options); _sync_rename()
def on_sel_none(_): channels_sel.value = tuple([]); _sync_rename()

def _sync_rename(*args):
    chs = list(channels_sel.value)
    try:
        current = json.loads(rename_area.value) if rename_area.value.strip() else {{}}
        if not isinstance(current, dict): current = {{}}
    except Exception:
        current = {{}}
    outmap = {{k: v for k, v in current.items() if k in chs}}
    for ch in chs: outmap.setdefault(ch, ch)
    rename_area.value = json.dumps(outmap, indent=2, ensure_ascii=False)

def on_save(_):
    try:
        folder = Path(STATE.get("folder", "."))
        sel_files = list(files_sel.value) or list(files_sel.options)
        parts = [Path(p).stem for p in sel_files]
        chs = list(channels_sel.value)
        mapping = json.loads(rename_area.value) if rename_area.value.strip() else {{}}
        mode = reref_mode.value
        custom = [x.strip() for x in reref_custom.value.split(",") if x.strip()] if mode == "custom" else []
        cfg_name = f"config. {{len(chs)}} chans"
        reref = {{"mode": mode}} | ({{"refs": custom}} if custom else {{}}
        )

        conf = {{}}
        for pid in parts:
            conf[pid] = {{"config": cfg_name, "remap": {{c: mapping.get(c, c) for c in chs}}, "reref": reref}}

        fname = save_name.value.strip() or "channel_config.json"
        path = folder / fname
        with open(path, "w", encoding="utf-8") as f:
            json.dump(conf, f, indent=2, ensure_ascii=False)

        preview.value = f"<pre>{{pretty_json_head(conf, 3)}}</pre>"
        with out:
            print("✅ Saved:", path)
    except Exception:
        with out:
            print(traceback.format_exc())

if _has_fc:
    chooser.register_callback(on_choose)
else:
    chooser.observe(lambda c: on_choose(), names="value")
btn_use_all.on_click(on_use_all)
btn_all.on_click(on_sel_all)
btn_none.on_click(on_sel_none)
channels_sel.observe(_sync_rename, names="value")
btn_save.on_click(on_save)

sdisplay(app)
